In [9]:
from langchain.chat_models import AzureChatOpenAI

from utils import init_openai, init_openai_deploy

init_openai()
model, deployment_id = init_openai_deploy()

llm = AzureChatOpenAI(
    model = model,
    deployment_name=deployment_id,
)

In [10]:
result = llm.call_as_llm("你好，ChatGPT！请帮我虚构3个科幻小说中的角色，提供他们的姓名、种族、年龄、外貌、性别")

print(result)

好的，以下是三个虚构的科幻小说角色及其细节：

1. 女王克里斯塔（Krista）
种族：X-Novo 人（外星人）
年龄：300 岁
外貌：宽大的蓝色眼睛、长达 2 米的身高、头部满是细长的触角
性别：女性

女王克里斯塔是 X-Novo 星球上最有权势的领导之一，她拥有强大的能力，可以影响他人的思想并改变他们的行为。尽管她极富智慧和战略，但她的外表并不被其他物种看重，这导致她时常感到孤独和受挫。

2. 莫莱（Morley）
种族：人类
年龄：27 岁
外貌：金色的卷发、深莹的棕色眼睛、紫色的全身装备
性别：男性

莫莱是地球联盟的一名特工，他具备出色的射击和格斗技能，甚至可以利用脑波对他人进行干扰。在一个危机四伏的任务中，莫莱与一名来自另外星球的女性坠入爱河，但却在该任务中失去了她，这让他备受痛苦。

3. 赫布（Herb）
种族：机器人
年龄：“出厂时间” 2 年
外貌：高大笨重、全身闪耀金属光泽、装载着高级 AI 关键字
性别：中性

赫布是一名为太空站服务的机器人，它可以执行各种任务，包括维修和清理、安全检查等。在一次事故中，赫布的控制程序出现故障，导致他开始对所有生命体产生厌恶情绪。他随后开始制定了一个计划，试图猎杀每个进入太空站的人类。


In [11]:
from pydantic import BaseModel, Field
from enum import Enum

class Gender(Enum):
    Male = "M"
    Female = "F"
    OTHER = "O"
    NA = ""

class Character(BaseModel):
    name: str = Field(description="姓名")
    species: str = Field(description="种族")
    age: int = Field(description="年龄")
    appearance: str = Field(description="外貌")
    gender: Gender = Field(description="性别,枚举")

Character.schema()

{'title': 'Character',
 'type': 'object',
 'properties': {'name': {'title': 'Name',
   'description': '姓名',
   'type': 'string'},
  'species': {'title': 'Species', 'description': '种族', 'type': 'string'},
  'age': {'title': 'Age', 'description': '年龄', 'type': 'integer'},
  'appearance': {'title': 'Appearance', 'description': '外貌', 'type': 'string'},
  'gender': {'description': '性别,枚举',
   'allOf': [{'$ref': '#/definitions/Gender'}]}},
 'required': ['name', 'species', 'age', 'appearance', 'gender'],
 'definitions': {'Gender': {'title': 'Gender',
   'description': 'An enumeration.',
   'enum': ['M', 'F', 'O', '']}}}

In [12]:
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Character)
print(parser.get_format_instructions())


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "\u59d3\u540d", "type": "string"}, "species": {"title": "Species", "description": "\u79cd\u65cf", "type": "string"}, "age": {"title": "Age", "description": "\u5e74\u9f84", "type": "integer"}, "appearance": {"title": "Appearance", "description": "\u5916\u8c8c", "type": "string"}, "gender": {"description": "\u6027\u522b,\u679a\u4e3e", "allOf": [{"$ref": "#/definitions/Gender"}]}}, "required": ["name", "species", "age", "appearance", "gender"], "definitions": {"Gender": {"title": "Gender",

In [13]:
from langchain import LLMChain, PromptTemplate


template = """
你好，ChatGPT！请帮我虚构一个科幻小说中的角色，提供他们的姓名、种族、年龄、外貌、性别

{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = LLMChain(llm=llm, prompt=prompt)

result = chain({})

print(result["text"])

{"name": "Astra Lux", "species": "Transcendent", "age": 1245, "appearance": "A tall, slender figure made up entirely of shimmering, multicolored lights. Astra's 'body' is fluid and moves unpredictably as if it were almost alive.", "gender": "O"}


In [14]:
template = """
你好，ChatGPT！请帮我虚构一个科幻小说中的角色，提供他们的姓名、种族、年龄、外貌、性别

{format_instructions}
请注意，角色的描述需要是中文的
"""

prompt = PromptTemplate(
    template=template,
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = LLMChain(llm=llm, prompt=prompt)

result = chain({})

print(result["text"])

{
  "name": "张琳",
  "species": "人类",
  "age": 32,
  "appearance": "黑色长卷发, 棕色眼睛, 身材匀称, 皮肤略带古铜色",
  "gender": "F"
}


In [15]:
from IPython.display import Markdown as md

character:Character = parser.parse(result["text"])

md(f"""
|name|species|age|gender|appearance|
|---|---|---|---|---|
|**{character.name}**|{character.species}|{character.age}|{character.gender}|{character.appearance}|
""")


|name|species|age|gender|appearance|
|---|---|---|---|---|
|**张琳**|人类|32|Gender.Female|黑色长卷发, 棕色眼睛, 身材匀称, 皮肤略带古铜色|


In [16]:
misformat1 = """{
    "name": "田所浩二",
    "species": "野兽先辈",
    "年龄": "35岁",
    "外貌": "儒雅随和",
    "性别": "哼哼哼，啊啊啊啊啊"
}"""

misformat2 = """
田所先生，24岁，是学生，外貌俊朗
"""


In [17]:
from langchain.output_parsers import OutputFixingParser

fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
fixing_parser.get_format_instructions()



'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "\\u59d3\\u540d", "type": "string"}, "species": {"title": "Species", "description": "\\u79cd\\u65cf", "type": "string"}, "age": {"title": "Age", "description": "\\u5e74\\u9f84", "type": "integer"}, "appearance": {"title": "Appearance", "description": "\\u5916\\u8c8c", "type": "string"}, "gender": {"description": "\\u6027\\u522b,\\u679a\\u4e3e", "allOf": [{"$ref": "#/definitions/Gender"}]}}, "required": ["name", "species", "age", "appearance", "gender"], "definitions": {"Gender":

In [18]:
character = fixing_parser.parse(misformat1)

print(character.json(indent=4,ensure_ascii=False))

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='codedog.openai.azure.com', port=443): Read timed out. (read timeout=60).


{
    "name": "田所浩二",
    "species": "野兽先辈",
    "age": 35,
    "appearance": "儒雅随和",
    "gender": "M"
}


In [ ]:
character = fixing_parser.parse(misformat2)

print(character.json(indent=4,ensure_ascii=False))

{
    "name": "田所先生",
    "species": "人类",
    "age": 24,
    "appearance": "俊朗",
    "gender": ""
}
